In [21]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator as RGI
from scipy.interpolate import LinearNDInterpolator as LNDI
from scipy.interpolate import RectBivariateSpline as RBS
from scipy.optimize import fsolve

#####################

nf = 100
nL = 16
nt = 6

fgrid = np.linspace(0,1,nf)            # table coordinates
fgrid[10] = 0.1055472493               # doldb
Lgrid = np.arange(nL, dtype=np.float64)
tgrid = np.arange(nt, dtype=np.float64)

#points = np.zeros((nf*nL*nt, 3))            # for LNDI
#for k in range(nt):                         # for LNDI
#    for j in range(nL):
#        for i in range(nf):
#            ip = k*nf*nL + j*nf + i
#            points[ip,0] = fgrid[i]
#            points[ip,1] = Lgrid[j]
#            points[ip,2] = tgrid[k]

c = np.zeros((nf, nL, nt))
h = np.zeros((nf, nL, nt))
T = np.zeros((nf, nL, nt))

###################################
###################################

for iL in range(nL):
    for it in range(nt):
        fname = "flm_" + str(iL).zfill(2) + '_' + str(it).zfill(2) + ".dat"
        data = np.loadtxt(fname)

        ii = interp1d(data[:,1], data[:,4])       # mixf, c
        c[:,iL, it] = ii(fgrid)

        ii = interp1d(data[:,1], data[:,3])       # mixf, h
        h[:,iL, it] = ii(fgrid)

        ii = interp1d(data[:,1], data[:,2])       # mixf, T
        T[:,iL, it] = ii(fgrid)

#c = c.reshape((nf*nL*nt,1))         # for LNDI
#h = h.reshape((nf*nL*nt,1))         # for LNDI
#T = T.reshape((nf*nL*nt,1))         # for LNDI

#####################

hmax = np.max(h)
hmin = np.min(h)
h = (h-hmin)/(hmax-hmin)

#####################

cI = RGI((fgrid, Lgrid, tgrid), c, bounds_error=False, fill_value=None, method='linear')      # call as cI([fvalue, Lvalue, tvalue])
hI = RGI((fgrid, Lgrid, tgrid), h, bounds_error=False, fill_value=None, method='linear')      
TI = RGI((fgrid, Lgrid, tgrid), T, bounds_error=False, fill_value=None, method='linear')

cII = RBS(Lgrid, tgrid, c[10,:,:])
hII = RBS(Lgrid, tgrid, h[10,:,:])
TII = RBS(Lgrid, tgrid, T[10,:,:])

#cI = LNDI(points, c)
#hI = LNDI(points, h)
#TI = LNDI(points, T)

######################

def get_Lt(f, h, c, Lt_guess=np.array([1.0,2.0])):

    def F(Lt):
        L = Lt[0]
        t = Lt[1]
        #F0 = hI([f, L, t])[0] - h
        #F1 = cI([f, L, t])[0] - c
        #F0 = hI(f,L,t)[0] - h            # for LNDI
        #F1 = cI(f,L,t)[0] - c            # for LNDI
        F0 = hII(L,t)[0][0] - h
        F1 = cII(L,t)[0][0] - c
        return np.array([F0, F1])

    Lt = fsolve(F, Lt_guess, xtol=1E-8, maxfev=1000, factor=1, epsfcn=1E-6)
    return Lt[0], Lt[1]

#####################

#odt = np.loadtxt('data_adia/dmp_00016.dat')    # 16, 23
odt = np.loadtxt('data_radi/dmp_00012.dat')     # 9, 10, 12
xodt = odt[:,0]
fodt = odt[:,8]
Todt = odt[:,7]
hodt = (odt[:, 28]-hmin)/(hmax-hmin)
codt = odt[:, 11] + odt[:,16] + odt[:,22] + odt[:,23]

fo = fodt[176]
ho = hodt[176]
co = codt[176]
To = Todt[176]

L,t = get_Lt(fo,ho,co)
print(L,t)
print(hII(L,t)-ho, cII(L,t)-co)

#####################

#nodt = len(fodt)
#TT = np.zeros(nodt)
#cc = np.zeros(nodt)
#hh = np.zeros(nodt)
#Lt_guess = np.array([1.0, 2.0])
#for i in range(nodt):
#    L, t = get_Lt(fodt[i], hodt[i], codt[i], Lt_guess)
#    TT[i] = TI([fodt[i], L, t])[0]
#    cc[i] = cI([fodt[i], L, t])[0]
#    hh[i] = hI([fodt[i], L, t])[0]
#    #TT[i] = TI([fodt[i], L, 0])[0]               # forces adiabatic 
#    #Lt_guess = np.array([L, t])
#    
#    #if i%10==0: print(L,t)
#    #if i%20==0: print(np.abs((codt[i]-cc[i])/codt[i]))
#    #if i%20==0: print(i, hI([fodt[i],L,t])[0]-hodt[i], cI([fodt[i],L,t])[0]-codt[i])
#    #if i%2==0: print(i, L, t, hI([fodt[i],L,t])[0], hodt[i], cI([fodt[i],L,t])[0],codt[i])
#
##------------------------
#
#plt.figure(figsize=(6,10))
#plt.subplot(2,1,1)
#plt.plot(xodt, codt, '-', lw=0.5)
#plt.plot(xodt, cc, 'o', ms=1, lw=0.5)
#plt.ylim([0,0.3])
##plt.xlim([-0.13,-0.1])
#plt.xlim([-0.25,0.2])
#
#plt.subplot(2,1,2)
#plt.plot(xodt, hodt, '-', lw=0.5)
#plt.plot(xodt, hh, 'o', ms=1, lw=0.5)
##plt.xlim([-0.13,-0.1])
#plt.xlim([-0.25,0.2])

1.2519162395927699 2.8044275166894095
[[-0.02620709]] [[-0.0101794]]


/var/folders/_7/42qnnz6j17g26jhlhv2l5_580000gn/T/ipykernel_88393/2581390386.py:90: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  Lt = fsolve(F, Lt_guess, xtol=1E-8, maxfev=1000, factor=1, epsfcn=1E-6)


In [12]:
fgrid

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747